# Variational Algorithms

## Table of Contents

0. [Variational Algorithms](#variational-algorithms)
1. [Simplified Hybrid Workflow](#simplified-hybrid-workflow)  
2. [Variational Theorem](#variational-theorem)  
   - [Mathematical Intuition for Energy and Ground States](#mathematical-intuition-for-energy-and-ground-states)  
   - [Variational Theorem of Quantum Mechanics](#variational-theorem-of-quantum-mechanics)  
3. [Summary](#summary)


## Variational Algorithms

This course covers the specifics of variational algorithms and near-term hybrid quantum-classical algorithms based on the variational theorem of quantum mechanics. These algorithms can leverage the utility provided by today's non-fault-tolerant quantum computers, making them ideal candidates to achieve quantum advantage.

Throughout this course, we will explore:

- Each step in the variational algorithm design workflow
- Trade-offs associated with each step
- How to use Qiskit Runtime primitives to optimize for speed and accuracy

While this course is meant to be a starting place for researchers and developers to explore the utility of quantum computers, feel free to explore the theoretical and foundational knowledge around quantum computing in general in the Basics of Quantum Information and Computation (also available as a series of YouTube videos).



## Simplified Hybrid Workflow

![alt text](../../../img/variational_workflow.svg)

Variational algorithms include several modular components that can be combined and optimized based on algorithm, software, and hardware advancements. This includes a *cost function* that describes a specific problem with a set of parameters, an *ansatz* to express the search space with these parameters, and an *optimizer* to iteratively explore the search space. During each iteration, the optimizer evaluates the cost function with the current parameters and selects the next iteration's parameters until it converges on an optimal solution. The hybrid nature of this family of algorithms comes from the fact that the cost functions are evaluated using quantum resources and optimized through classical ones.

1. **Initialize problem**: Variational algorithms start by initializing the quantum computer in a default state $|0\rangle$, then transforming it to some desired (non-parametrized) state $|\rho\rangle$, which we will call *reference state*. 
   This transformation is represented by the application of a unitary reference operator $U_R$ on the default state, such that $U_R |0\rangle = |\rho\rangle$.

2. **Prepare ansatz**: To begin iteratively optimizing from the default state $|0\rangle$ to the target state $|\psi(\vec{\theta})\rangle$, we must define a *variational form* $U_V(\vec{\theta})$ to represent a collection of parametrized states for our variational algorithm to explore.
   We refer to any particular combination of reference state and variational form as an ansatz, such that: $U_A(\vec{\theta}) := U_V(\vec{\theta})U_R$.
   Ansatz will ultimately take the form of parametrized quantum circuits capable of taking the default state $|0\rangle$ to the target state $|\psi(\vec{\theta})\rangle$.
   All in all we will have:

   $$|0\rangle \xrightarrow{U_R} U_R|0\rangle = |\rho\rangle \xrightarrow{U_V(\vec{\theta})} U_A(\vec{\theta})|0\rangle$$
   $$= U_V(\vec{\theta})U_R|0\rangle$$
   $$= U_V(\vec{\theta})|\rho\rangle$$
   $$= |\psi(\vec{\theta})\rangle$$

3. **Evaluate cost function**: We can encode our problem into a cost function $C(\vec{\theta})$ as a linear combination of Pauli operators, run on a quantum system. While this can be information about physical systems, such as energy or spin, we can also encode non-physical problems as well. We can leverage Qiskit Runtime primitives to address noise with error suppression and mitigation while evaluating our cost function.

4. **Optimize parameters**: Evaluations are taken to a classical computer, where a classical optimizer analyzes them and chooses the next set of values for the variational parameters. If we have a pre-existing optimal solution, we can set it as an *initial point* $\vec{\theta_0}$ to bootstrap our optimization. Using this initial state $|\psi(\vec{\theta_0})\rangle$ could help our optimizer find a valid solution faster.

5. **Adjust ansatz parameters with results, and re-run**: the entire process is repeated until the classical optimizer's finalization criteria are met, and an optimal set of parameter values $\vec{\theta^*}$ is returned. The proposed solution state for our problem will then be $|\psi(\vec{\theta^*})\rangle = U_A(\vec{\theta^*})|0\rangle$.


## Variational Theorem

A common goal of variational algorithms is to find the quantum state with the lowest or highest eigenvalue of a certain observable. A key insight we'll use is the *variational theorem* of quantum mechanics. Before going into its full statement, let us explore some of the mathematical intuition behind it.

### Mathematical Intuition for Energy and Ground States


In quantum mechanics, energy comes in the form of a quantum observable usually referred to as the Hamiltonian, which we'll denote by $H$. Let us consider its spectral decomposition:

$$H = \sum_{k=0}^{N-1} \lambda_k |\phi_k\rangle\langle\phi_k|$$

where $N$ is the dimensionality of the space of states, $\lambda_k$ is the $k$-th eigenvalue or, physically, the $k$-th energy level, and $|\phi_k\rangle$ is the corresponding eigenstate: $H|\phi_k\rangle = \lambda_k|\phi_k\rangle$, the expected energy of a system in the (normalized) state $|\psi\rangle$ will be:

$$\langle\psi|H|\psi\rangle = \langle\psi|\left(\sum_{k=0}^{N-1} \lambda_k|\phi_k\rangle\langle\phi_k|\right)|\psi\rangle$$

$$= \sum_{k=0}^{N-1} \lambda_k\langle\psi|\phi_k\rangle\langle\phi_k|\psi\rangle$$

$$= \sum_{k=0}^{N-1} \lambda_k|\langle\psi|\phi_k\rangle|^2$$

If we take into account that $\lambda_0 \leq \lambda_k, \forall k$, we have:

$$\langle\psi|H|\psi\rangle = \sum_{k=0}^{N-1} \lambda_k|\langle\psi|\phi_k\rangle|^2$$

$$\geq \sum_{k=0}^{N-1} \lambda_0|\langle\psi|\phi_k\rangle|^2$$

$$= \lambda_0 \sum_{k=0}^{N-1} |\langle\psi|\phi_k\rangle|^2$$

$$= \lambda_0$$

Since $\{|\phi_k\rangle\}_{k=0}^{N-1}$ is an orthonormal basis, the probability of measuring $|\phi_k\rangle$ is $p_k = |\langle\psi|\phi_k\rangle|^2$, and the sum of all probabilities is such that $\sum_{k=0}^{N-1} |\langle\psi|\phi_k\rangle|^2 = \sum_{k=0}^{N-1} p_k = 1$. In short, the expected energy of any system is higher than the lowest energy or ground state energy:

$$\langle\psi|H|\psi\rangle \geq \lambda_0.$$

The above argument applies to any valid (normalized) quantum state $|\psi\rangle$, so it is perfectly possible to consider parametrized states $|\psi(\vec{\theta})\rangle$ that depend on a parameter vector $\vec{\theta}$. This is where the "variational" part comes into play. If we consider a cost function given by $C(\vec{\theta}) := \langle\psi(\vec{\theta})|H|\psi(\vec{\theta})\rangle$ and want to minimize it, the minimum will always satisfy:

$$\min_{\vec{\theta}} C(\vec{\theta}) = \min_{\vec{\theta}} \langle\psi(\vec{\theta})|H|\psi(\vec{\theta})\rangle \geq \lambda_0.$$

The minimum value of $C(\vec{\theta})$ will be the closest that one can get to $\lambda_0$ using the parametrized states $|\psi(\vec{\theta})\rangle$, and equality will only be reached if there exists a parameter vector $\vec{\theta}^*$ such that $|\psi(\vec{\theta}^*)\rangle = |\phi_0\rangle$



### Variational Theorem of Quantum Mechanics

If the (normalized) state $|\psi\rangle$ of a quantum system depends on a parameter vector $\vec{\theta}$, then the optimal approximation of the ground state (i.e. the eigenstate $|\phi_0\rangle$ with the minimum eigenvalue $\lambda_0$) is the one that minimizes the expectation value of the Hamiltonian $\hat{\mathcal{H}}$:

$$
\langle \hat{\mathcal{H}} \rangle (\vec{\theta}) := \langle \psi(\vec{\theta}) | \hat{\mathcal{H}} | \psi(\vec{\theta}) \rangle \geq \lambda_0
$$

The reason why the variational theorem is stated in terms of energy minima is that it includes a number of mathematical assumptions:

- For physical reasons, a finite lower bound to the energy $E \geq \lambda_0 > -\infty$ needs to exist, even for $N \to \infty$.
- Upper bounds do not generally exist.

However, mathematically speaking, there is nothing special about the Hamiltonian $\hat{\mathcal{H}}$ beyond these assumptions, so the theorem can be generalized to other quantum observables and their eigenstates provided they follow the same constraints. Also, note that if finite upper bounds exist, the same mathematical arguments could be made for maximizing eigenvalues by swapping lower bounds for upper bounds.



## Summary

With this lesson, you learned the high-level view of variational algorithms. Over the following lessons, we'll explore each step in greater detail, and their associated trade-offs.